<a href="https://colab.research.google.com/github/KenObata/TISMIR_notebooks/blob/main/week15_XLnet_baseline_keras_768_batch_normaliza_every_layer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Week15: This notebook uses Pre-Trained word matrix-> SMOTE -> undersample by EDA -> append to word vectors. -> using Word2Vec. After that I apply SMOTE to balance out. 
Added task: Grid Search for best parameter in SVM.

Situation: English only (=multi-class).
Split: StratifiedKfold.
Reference: https://github.com/jasonwei20/eda_nlp


### set up

In [ ]:
!pip3 install scikit-multilearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 89 kB 5.6 MB/s 


In [ ]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import sklearn
from collections import Counter

from skmultilearn.model_selection import IterativeStratification   
from sklearn.model_selection import StratifiedKFold
from scipy.sparse import csr_matrix

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import balanced_accuracy_score
DIR = '/content/drive/MyDrive/music4all/'
def get_balanced_accuracy(model, McNemar, is_fine_tuning, drop_out_rate,learning_rate_transfer_learning, learning_rate_fine_tuning):
  test_y = test.map(map_func_only_y)
  y_category=np.zeros((TEST_SIZE, ))
  counter=0
  for label_tensor in test_y.take(len(test_y)):
    y_test = np.argmax(label_tensor, axis=1)
    for label in y_test:
      y_category[counter]=label
      counter+=1

  X_test, y_test = test.map(map_func_only_X), y_category
  y_predict_test = np.asarray(model.predict(X_test))
  y_predict_test = np.argmax(y_predict_test, axis=1)
  print(classification_report(y_test, y_predict_test) )
  print(balanced_accuracy_score(y_test, y_predict_test))

  McNemar[(is_fine_tuning, drop_out_rate,learning_rate_transfer_learning, learning_rate_fine_tuning)] = []
  for ground_truh, pred in zip(y_test, y_predict_test):
        if ground_truh==pred:
          McNemar[(is_fine_tuning, drop_out_rate,learning_rate_transfer_learning, learning_rate_fine_tuning)].append(True)
        else:
          McNemar[(is_fine_tuning, drop_out_rate,learning_rate_transfer_learning, learning_rate_fine_tuning)].append(False)
  with open(DIR+ "XLnet_1024_simple_dropout_and_batch_normalization_log.txt", "a") as f:
    print("======================================", file=f)
    print("is_fine_tuning?:", is_fine_tuning, "drop_out_rate: ", drop_out_rate, "learning_rate_transfer_learning: ", learning_rate_transfer_learning,
          "learning_rate_fine_tuning: ", learning_rate_fine_tuning, file=f)
    print(classification_report(y_test, y_predict_test) , file=f)
    print(balanced_accuracy_score(y_test, y_predict_test), file=f)

  return balanced_accuracy_score(y_test, y_predict_test), McNemar

### Data Preparation(Kfold split)

Create dataframe for Gensim

In [ ]:
import pandas as pd

DIR = '/content/drive/MyDrive/music4all/'
df_genre_by_lang = pd.read_csv(DIR + 'df_genre_by_lang_full.csv')
df_genre_by_lang

,Unnamed: 0,id,genres,lang,lyric,number_of_line
0,0,0009fFIM1eYThaPg,pop,en,a sunny day so I got nowhere to hide Not a clo...,91
1,1,00P2bHdWFkghmDqz,soul,en,Tell me a tale that always was Sing me a song ...,36
2,2,00b6fV3nx5z2b8Ls,pop,en,A buh A buh You went to school to learn girl T...,74
3,3,013QDoTqbexEwkHr,pop,en,like a conversation where stops to breathe Is ...,20
4,4,01EKNot8qVgZpKM7,rock,en,Say the words I cannot say Say them on another...,31
...,...,...,...,...,...,...
13535,13535,zzT504Z94j1IAuc3,indie rock,en,think what afraid of come in you know been mad...,18
13536,13536,zzgS4ZqyswamEWNj,pop,en,Oh yeah yeah Last night I took a walk in the s...,75
13537,13537,zzx8CWdM7qkxKQpC,indie rock,en,Innocence it come easy in a sense it never wil...,34
13538,13538,zzz0n04uuTUA7fNh,pop,en,Girl you know how I feel I really Since you be...,65


In [ ]:
def load_data(df_col, y):
    texts, labels = [], []
    
    for line in df_col:
        # texts are already tokenized, just split on space
        # in a real use-case we would put more effort in preprocessing
        texts.append(line.split(' '))
    return pd.DataFrame({'texts': texts, 'labels': y})

In [ ]:
data = load_data(df_genre_by_lang["lyric"], df_genre_by_lang["genres"])

In [ ]:
data

,texts,labels
0,"[a, sunny, day, so, I, got, nowhere, to, hide,...",pop
1,"[Tell, me, a, tale, that, always, was, Sing, m...",soul
2,"[A, buh, A, buh, You, went, to, school, to, le...",pop
3,"[like, a, conversation, where, stops, to, brea...",pop
4,"[Say, the, words, I, cannot, say, Say, them, o...",rock
...,...,...
13535,"[think, what, afraid, of, come, in, you, know,...",indie rock
13536,"[Oh, yeah, yeah, Last, night, I, took, a, walk...",pop
13537,"[Innocence, it, come, easy, in, a, sense, it, ...",indie rock
13538,"[Girl, you, know, how, I, feel, I, really, Sin...",pop


In [ ]:
data['labels'] = data['labels'].astype('category')
label_mapping = data['labels'].cat.categories
data['labels'] = data['labels'].cat.codes
X = data['texts']
y = data['labels']

In [ ]:
type(X)

pandas.core.series.Series

In [ ]:
def StratifiedKFold_feature_and_df_glove(df, feature_list, y_name):
  skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1209)  # 20% for test set 
  y = df[y_name]
  skf.get_n_splits(df[ feature_list ], y)

  splits = []

  for train_index, test_index in skf.split(df[ feature_list ], y):
      print("TRAIN:", train_index, "TEST:", test_index)
      X_train, X_test = df[ feature_list ].loc[train_index], df[ feature_list ].loc[test_index]
      y_train, y_test = y.loc[train_index], y.loc[test_index]
      splits.append({'X_train': X_train, 'X_test': X_test, 'y_train':y_train, 'y_test':y_test })
  return splits

In [ ]:
def StratifiedKFold_feature_and_df(X, y):
  skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1209)  # 20% for test set 
  #y = df[y_name]
  skf.get_n_splits(X, y)#df[ feature_list ]

  splits = []

  for train_index, test_index in skf.split(X, y):#df[ feature_list ]
      print("TRAIN:", train_index, "TEST:", test_index)
      X_train, X_test = X.loc[train_index], X.loc[test_index]
      y_train, y_test = y.loc[train_index], y.loc[test_index]
      splits.append({'X_train': X_train, 'X_test': X_test, 'y_train':y_train, 'y_test':y_test })
  return splits

In [ ]:
#feature_list = ["texts"] #this is BOW and TF-IDF
#splits = StratifiedKFold_feature_and_df( data, feature_list, 'labels')
splits = StratifiedKFold_feature_and_df( X,y)

TRAIN: [    0     1     3 ... 13537 13538 13539] TEST: [    2     4     5 ... 13526 13532 13535]
TRAIN: [    0     2     4 ... 13535 13536 13539] TEST: [    1     3     7 ... 13530 13537 13538]
TRAIN: [    0     1     2 ... 13537 13538 13539] TEST: [    8    14    22 ... 13521 13531 13536]
TRAIN: [    0     1     2 ... 13537 13538 13539] TEST: [   10    12    15 ... 13523 13525 13534]
TRAIN: [    1     2     3 ... 13536 13537 13538] TEST: [    0     6    11 ... 13529 13533 13539]


In [ ]:
len(splits)

5

In [ ]:
split0=splits[0]
print(split0['X_train'].shape)
print(split0['y_train'].shape)
print(split0['X_test'].shape)
print(split0['y_test'].shape)

(10832,)
(10832,)
(2708,)
(2708,)


In [ ]:
split0['X_train']

0        [a, sunny, day, so, I, got, nowhere, to, hide,...
1        [Tell, me, a, tale, that, always, was, Sing, m...
3        [like, a, conversation, where, stops, to, brea...
6        [Locked, up, tight, Like, I, would, never, fee...
7        [sittin, in, the, crib, dreamin, about, leer, ...
                               ...                        
13534    [grandma, cookies, nigga, Shout, out, to, fron...
13536    [Oh, yeah, yeah, Last, night, I, took, a, walk...
13537    [Innocence, it, come, easy, in, a, sense, it, ...
13538    [Girl, you, know, how, I, feel, I, really, Sin...
13539    [wwI, oh, must, go, on, standing, You, break, ...
Name: texts, Length: 10832, dtype: object

In [ ]:
split0['y_train']

0        4
1        9
3        4
6        4
7        6
        ..
13534    6
13536    4
13537    3
13538    4
13539    8
Name: labels, Length: 10832, dtype: int8

### Use my self programmed balanced accuracy

In [ ]:
metrics = Metrics()
history = model.fit(train, validation_data=val, epochs=10, class_weight=my_weight ,callbacks=[metrics])
metrics.get_data()

Epoch 1/10
339/339 [==============================] - 225s 645ms/step - loss: 0.5565 - categorical_accuracy: 0.3878 - val_loss: 1.9337 - val_categorical_accuracy: 0.3273
Epoch 2/10
339/339 [==============================] - 218s 645ms/step - loss: 0.5397 - categorical_accuracy: 0.3626 - val_loss: 2.0651 - val_categorical_accuracy: 0.2764
Epoch 3/10
339/339 [==============================] - 218s 645ms/step - loss: 0.5375 - categorical_accuracy: 0.3392 - val_loss: 2.0931 - val_categorical_accuracy: 0.2273
Epoch 4/10
339/339 [==============================] - 218s 645ms/step - loss: 0.5378 - categorical_accuracy: 0.3197 - val_loss: 2.0576 - val_categorical_accuracy: 0.2459
Epoch 5/10
339/339 [==============================] - 219s 645ms/step - loss: 0.5333 - categorical_accuracy: 0.3077 - val_loss: 2.0292 - val_categorical_accuracy: 0.2816
Epoch 6/10
339/339 [==============================] - 219s 646ms/step - loss: 0.5373 - categorical_accuracy: 0.2967 - val_loss: 2.1738 - val_categoric

[{'balanced_accuracy': 0.09744849916749872},
 {'balanced_accuracy': 0.10420636932765996},
 {'balanced_accuracy': 0.10970405736872939},
 {'balanced_accuracy': 0.10356561320774078},
 {'balanced_accuracy': 0.10792864804613309},
 {'balanced_accuracy': 0.09168989857066105},
 {'balanced_accuracy': 0.09926135499834263},
 {'balanced_accuracy': 0.10014630125778516},
 {'balanced_accuracy': 0.09807505457274496},
 {'balanced_accuracy': 0.09881853716030842}]

## From here, separate X_train, X_test from KFoldSplit

### Preprocess my lyrics data (Official train and test)

In [ ]:
import tensorflow as tf
!pip3 install transformers
SEQ_LEN = 1024#256#512

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 11.7 MB/s 
     |████████████████████████████████| 120 kB 60.2 MB/s 
     |████████████████████████████████| 6.6 MB 52.5 MB/s 


In [ ]:
split0['X_test']

2        [A, buh, A, buh, You, went, to, school, to, le...
4        [Say, the, words, I, cannot, say, Say, them, o...
5        [I, was, alone, I, was, made, of, stone, You, ...
9        [Again, the, burden, of, losing, rests, upon, ...
20       [only, been, three, weeks, And, a, bag, of, sp...
                               ...                        
13517    [Like, the, legend, of, the, Phoenix, All, end...
13522    [Mr, Telephone, man, something, wrong, with, m...
13526    [can, you, imagine, what, it, would, be, like,...
13532    [Love, of, my, life, hurt, me, broken, my, hea...
13535    [think, what, afraid, of, come, in, you, know,...
Name: texts, Length: 2708, dtype: object

In [ ]:
def prepare_lyrics(X_series):
  for i, token_list in X_series.items():
    if type(token_list) is list:
      X_series.loc[i] = ' '.join(token_list)
  return X_series

In [ ]:
split0['X_test'] = prepare_lyrics(split0['X_test'] )
split0['X_test']

2        A buh A buh You went to school to learn girl T...
4        Say the words I cannot say Say them on another...
5        I was alone I was made of stone You took me ho...
9        Again the burden of losing rests upon my shoul...
20       only been three weeks And a bag of speed from ...
                               ...                        
13517    Like the legend of the Phoenix All ends with b...
13522    Mr Telephone man something wrong with my line ...
13526    can you imagine what it would be like we never...
13532    Love of my life hurt me broken my heart and no...
13535    think what afraid of come in you know been mad...
Name: texts, Length: 2708, dtype: object

In [ ]:
split0['X_train'] = prepare_lyrics(split0['X_train'] )
split0['X_train'] 

0        a sunny day so I got nowhere to hide Not a clo...
1        Tell me a tale that always was Sing me a song ...
3        like a conversation where stops to breathe Is ...
6        Locked up tight Like I would never feel again ...
7        sittin in the crib dreamin about leer jets and...
                               ...                        
13534    grandma cookies nigga Shout out to fronto leaf...
13536    Oh yeah yeah Last night I took a walk in the s...
13537    Innocence it come easy in a sense it never wil...
13538    Girl you know how I feel I really Since you be...
13539    wwI oh must go on standing You break that whic...
Name: texts, Length: 10832, dtype: object

In [ ]:
split0['y_train'] 

0        4
1        9
3        4
6        4
7        6
        ..
13534    6
13536    4
13537    3
13538    4
13539    8
Name: labels, Length: 10832, dtype: int8

use ALBERT
Be careful, you need sentencepiece library:https://stackoverflow.com/questions/65854722/huggingface-albert-tokenizer-nonetype-error-with-colab

In [ ]:
!pip3 install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 11.9 MB/s 


In [ ]:
def map_func(input_ids, masks, labels):
  return {'input_ids': input_ids, 'attention_mask':masks}, labels

339 because 10800/32

In [ ]:
SPLIT = 0.9

In [ ]:
len(split0['X_train'])-int(len(split0['X_train'])*0.9)

1084

In [ ]:
def map_func_only_X(val_dictionary, labels):
  return {'input_ids': val_dictionary['input_ids'], 'attention_mask':val_dictionary['attention_mask']}

In [ ]:
def map_func_only_y(val_dictionary, labels):
  return labels

In [ ]:
VAL_SIZE=1088

In [ ]:
counter = Counter(split0['y_train'])
SUM=0
for item in list(counter.values()) :
  SUM+=item
#SUM = sum(counter.values())
SUM

10832

In [ ]:
#Tutorial
#weight_for_0 = (1 / neg) * (total / 2.0)
#weight_for_1 = (1 / pos) * (total / 2.0)

counter = Counter(split0['y_train'])
my_weight2 = {}
print(counter)

for genre in counter:
  #print(genre, counter[genre])
  my_weight2[genre] = (1/counter[genre]) * (SUM/10)
my_weight2

Counter({4: 4143, 7: 1159, 9: 1030, 3: 865, 6: 783, 0: 763, 1: 690, 8: 556, 2: 537, 5: 306})


{4: 0.2614530533429882,
 9: 1.051650485436893,
 6: 1.383397190293742,
 2: 2.0171322160148977,
 0: 1.4196592398427261,
 7: 0.9345987920621226,
 1: 1.569855072463768,
 5: 3.539869281045752,
 8: 1.948201438848921,
 3: 1.2522543352601156}

### Parameter tuning

In [ ]:
TEST_SIZE=len(split0['y_test'])
TEST_SIZE

2708

In [ ]:
from transformers import XLNetLMHeadModel, XLNetTokenizer
import numpy as np

split0=splits[0]
split0['X_train'] = prepare_lyrics(split0['X_train'] )
split0['X_test'] = prepare_lyrics(split0['X_test'] )

SEQ_LEN=768
SEQ_LEN2=768
print(split0['X_train'].shape, split0['X_test'].shape)

tokenizer = XLNetTokenizer.from_pretrained("xlnet-base-cased")

Xids_train = np.zeros((split0['X_train'].shape[0], SEQ_LEN))
Xmask_train = np.zeros((split0['X_train'].shape[0], SEQ_LEN))
Xids_test = np.zeros((split0['X_test'].shape[0], SEQ_LEN))
Xmask_test = np.zeros((split0['X_test'].shape[0], SEQ_LEN))

for i, lyric in enumerate(split0['X_train']):
  tokens = tokenizer.encode_plus(lyric, max_length=SEQ_LEN, truncation =True, padding="max_length", add_special_tokens = True, return_token_type_ids= False, return_attention_mask = True, return_tensors= 'tf')
  Xids_train[i,:], Xmask_train[i,:] = tokens['input_ids'], tokens['attention_mask']

for i, lyric in enumerate(split0['X_test']):
  tokens_test = tokenizer.encode_plus(lyric, max_length=SEQ_LEN, truncation =True, padding="max_length"
    , add_special_tokens = True, return_token_type_ids= False, return_attention_mask = True, return_tensors= 'tf')
  Xids_test[i,:], Xmask_test[i,:] = tokens_test['input_ids'], tokens_test['attention_mask']

print("Xids_train.shape, Xids_test.shape: ",Xids_train.shape, Xids_test.shape)

labels_train = np.zeros((split0['y_train'].shape[0], 10))
labels_train[ np.arange(split0['y_train'].shape[0]), split0['y_train'].values] =1
labels_test = np.zeros((split0['y_test'].shape[0], 10))
labels_test[ np.arange(split0['y_test'].shape[0]), split0['y_test'].values] =1

dataset_train = tf.data.Dataset.from_tensor_slices((Xids_train, Xmask_train, labels_train))
dataset_test = tf.data.Dataset.from_tensor_slices((Xids_test, Xmask_test, labels_test))

dataset_train = dataset_train.map(map_func)
dataset_test = dataset_test.map(map_func)
dataset_train = dataset_train.shuffle(42).batch(4)

DS_LEN=len(dataset_train)
SPLIT=0.9
train = dataset_train.take(round(DS_LEN*SPLIT))
val = dataset_train.skip(round(DS_LEN*SPLIT))
test = dataset_test.batch(4)

(10832,) (2708,)


Xids_train.shape, Xids_test.shape:  (10832, 768) (2708, 768)


In [ ]:
McNemar={}

In [ ]:
from transformers import TFXLNetModel


#drop_out_rates = [0.1, 0.2, 0.3,0.4, 0.5]
balanced_accuracies_transfer_learning = []
balanced_accuracies_fine_tuning = []

drop_out_rate = 0.2
learning_rate_transfer_learnings = [1e-3, 1e-4]
learning_rate_fine_tunings = [1e-5, 1e-6]

for learning_rate_transfer_learning in learning_rate_transfer_learnings:
  for learning_rate_fine_tuning in learning_rate_fine_tunings:
    print("learning_rate_transfer_learning: ",learning_rate_transfer_learning, "learning_rate_fine_tuning: "
    , learning_rate_fine_tuning, "drop_out_rate:", drop_out_rate)
    #step1
    #bert_base = TFAutoModel.from_pretrained('bert-base-cased')
    xlnet_base = TFXLNetModel.from_pretrained('xlnet-base-cased')
    input_ids = tf.keras.layers.Input(shape=(SEQ_LEN2,), name= 'input_ids', dtype='int32')
    mask = tf.keras.layers.Input(shape=(SEQ_LEN2,), name='attention_mask')

    embeddings = xlnet_base(input_ids, attention_mask= mask)[0]
    X = tf.keras.layers.GlobalMaxPool1D()(embeddings)
    X = tf.keras.layers.BatchNormalization()(X)
    X = tf.keras.layers.Dense(128, activation = 'relu')(X)
    X = tf.keras.layers.BatchNormalization()(X)#added
    X = tf.keras.layers.Dropout(drop_out_rate)(X)
    X = tf.keras.layers.Dense(32, activation = 'relu')(X)
    X = tf.keras.layers.BatchNormalization()(X)#added
    X = tf.keras.layers.Dropout(drop_out_rate)(X)
    y= tf.keras.layers.Dense(10, activation = 'softmax' , name= 'outputs')(X)

    model1 = tf.keras.Model(inputs=[input_ids, mask], outputs=y)
    model1.layers[2].trainable = False
    print(model1.summary())

    #step2
    optimizer = tf.keras.optimizers.Adam(learning_rate_transfer_learning)
    loss = tf.keras.losses.CategoricalCrossentropy(name='categorical_crossentropy')#from_logits=False,label_smoothing=0.0,axis=-1,
    metrics = []
    metrics.append(
        tf.metrics.CategoricalAccuracy(name='categorical_accuracy', dtype=None))
    model1.compile(optimizer=optimizer, loss= loss, metrics=metrics)
    #model2.summary() #Check trainable params increased.

    #step3: transfer learning
    checkpoint_filepath = DIR + 'checkpoints'
    model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_filepath, save_weights_only=True
                                                  , monitor='val_categorical_accuracy', save_best_only=False)#, mode='max'
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_categorical_accuracy', patience=3)
    history = model1.fit(train, validation_data=val, epochs=6, class_weight=my_weight2 ,callbacks=[ early_stopping, model_checkpoint_callback])

    #step4: predict
    balanced_acc, McNemar=get_balanced_accuracy(model1, McNemar, False, drop_out_rate,learning_rate_transfer_learning, learning_rate_fine_tuning )
    balanced_accuracies_transfer_learning.append( balanced_acc )

    #step5: fine tune
    print("Fine tuning---------------")
    model1.layers[2].trainable = True

    # It's important to recompile your model after you make any changes
    optimizer = tf.keras.optimizers.Adam(learning_rate_fine_tuning)
    loss = tf.keras.losses.CategoricalCrossentropy(name='categorical_crossentropy')#from_logits=False,label_smoothing=0.0,axis=-1,
    metrics = []
    metrics.append(
        tf.metrics.CategoricalAccuracy(name='categorical_accuracy', dtype=None))
    model1.compile(optimizer=optimizer, loss= loss, metrics=metrics)

    history = model1.fit(train, validation_data=val, epochs=6, class_weight=my_weight2 ,callbacks=[ early_stopping])
    balanced_acc, McNemar=get_balanced_accuracy(model1, McNemar, True, drop_out_rate,learning_rate_transfer_learning, learning_rate_fine_tuning )
    balanced_accuracies_fine_tuning.append( balanced_acc )
    print("----------------------------------------")
    del(model1)


learning_rate_transfer_learning:  0.001 learning_rate_fine_tuning:  1e-05 drop_out_rate: 0.2


Some layers from the model checkpoint at xlnet-base-cased were not used when initializing TFXLNetModel: ['lm_loss']
- This IS expected if you are initializing TFXLNetModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFXLNetModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFXLNetModel were initialized from the model checkpoint at xlnet-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFXLNetModel for predictions without further training.


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 768)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 768)]        0           []                               
                                                                                                  
 tfxl_net_model (TFXLNetModel)  TFXLNetModelOutput(  116718336   ['input_ids[0][0]',              
                                last_hidden_state=(               'attention_mask[0][0]']         
                                None, 768, 768),                                                  
                                 mems=((768, None,                                            

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classificat

              precision    recall  f1-score   support

         0.0       0.14      0.08      0.11       190
         1.0       0.21      0.09      0.13       173
         2.0       0.10      0.01      0.01       135
         3.0       0.00      0.00      0.00       216
         4.0       0.61      0.05      0.09      1036
         5.0       0.09      0.04      0.06        76
         6.0       0.09      0.99      0.17       195
         7.0       0.00      0.00      0.00       290
         8.0       1.00      0.01      0.01       139
         9.0       0.16      0.15      0.16       258

    accuracy                           0.12      2708
   macro avg       0.24      0.14      0.07      2708
weighted avg       0.34      0.12      0.08      2708

0.14231380435725033
Fine tuning---------------
Epoch 1/6


...

              precision    recall  f1-score   support

         0.0       0.10      0.20      0.14       190
         1.0       0.33      0.20      0.25       173
         2.0       0.00      0.00      0.00       135
         3.0       0.23      0.07      0.11       216
         4.0       0.63      0.04      0.07      1036
         5.0       0.07      0.30      0.11        76
         6.0       0.14      0.97      0.24       195
         7.0       0.00      0.00      0.00       290
         8.0       0.19      0.17      0.18       139
         9.0       0.24      0.26      0.25       258

    accuracy                           0.16      2708
   macro avg       0.19      0.22      0.14      2708
weighted avg       0.33      0.16      0.11      2708

0.22201906086712864
----------------------------------------
learning_rate_transfer_learning:  0.0001 learning_rate_fine_tuning:  1e-05 drop_out_rate: 0.2


Some layers from the model checkpoint at xlnet-base-cased were not used when initializing TFXLNetModel: ['lm_loss']
- This IS expected if you are initializing TFXLNetModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFXLNetModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFXLNetModel were initialized from the model checkpoint at xlnet-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFXLNetModel for predictions without further training.


Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 768)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 768)]        0           []                               
                                                                                                  
 tfxl_net_model_2 (TFXLNetModel  TFXLNetModelOutput(  116718336  ['input_ids[0][0]',              
 )                              last_hidden_state=(               'attention_mask[0][0]']         
                                None, 768, 768),                                                  
                                 mems=((768, None,                                          

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classificat

2000/2437 [=======================>......] - ETA: 8:28 - loss: 2.3144 - categorical_accuracy: 0.1700

In [ ]:
from transformers import TFXLNetModel


#drop_out_rates = [0.1, 0.2, 0.3,0.4, 0.5]
balanced_accuracies_transfer_learning = []
balanced_accuracies_fine_tuning = []

drop_out_rate = 0.3
learning_rate_transfer_learnings = [1e-3, 1e-4]
learning_rate_fine_tunings = [1e-5, 1e-6]

for learning_rate_transfer_learning in learning_rate_transfer_learnings:
  for learning_rate_fine_tuning in learning_rate_fine_tunings:
    print("learning_rate_transfer_learning: ",learning_rate_transfer_learning, "learning_rate_fine_tuning: "
    , learning_rate_fine_tuning, "drop_out_rate:", drop_out_rate)
    #step1
    #bert_base = TFAutoModel.from_pretrained('bert-base-cased')
    xlnet_base = TFXLNetModel.from_pretrained('xlnet-base-cased')
    input_ids = tf.keras.layers.Input(shape=(SEQ_LEN2,), name= 'input_ids', dtype='int32')
    mask = tf.keras.layers.Input(shape=(SEQ_LEN2,), name='attention_mask')

    embeddings = xlnet_base(input_ids, attention_mask= mask)[0]
    X = tf.keras.layers.GlobalMaxPool1D()(embeddings)
    X = tf.keras.layers.BatchNormalization()(X)
    X = tf.keras.layers.Dense(128, activation = 'relu')(X)
    X = tf.keras.layers.BatchNormalization()(X)#added
    X = tf.keras.layers.Dropout(drop_out_rate)(X)
    X = tf.keras.layers.Dense(32, activation = 'relu')(X)
    X = tf.keras.layers.BatchNormalization()(X)#added
    X = tf.keras.layers.Dropout(drop_out_rate)(X)
    y= tf.keras.layers.Dense(10, activation = 'softmax' , name= 'outputs')(X)

    model1 = tf.keras.Model(inputs=[input_ids, mask], outputs=y)
    model1.layers[2].trainable = False
    print(model1.summary())

    #step2
    optimizer = tf.keras.optimizers.Adam(learning_rate_transfer_learning)
    loss = tf.keras.losses.CategoricalCrossentropy(name='categorical_crossentropy')#from_logits=False,label_smoothing=0.0,axis=-1,
    metrics = []
    metrics.append(
        tf.metrics.CategoricalAccuracy(name='categorical_accuracy', dtype=None))
    model1.compile(optimizer=optimizer, loss= loss, metrics=metrics)
    #model2.summary() #Check trainable params increased.

    #step3: transfer learning
    checkpoint_filepath = DIR + 'checkpoints'
    model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_filepath, save_weights_only=True
                                                  , monitor='val_categorical_accuracy', save_best_only=False)#, mode='max'
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_categorical_accuracy', patience=3)
    history = model1.fit(train, validation_data=val, epochs=6, class_weight=my_weight2 ,callbacks=[ early_stopping, model_checkpoint_callback])

    #step4: predict
    balanced_acc, McNemar=get_balanced_accuracy(model1, McNemar, False, drop_out_rate,learning_rate_transfer_learning, learning_rate_fine_tuning )
    balanced_accuracies_transfer_learning.append( balanced_acc )

    #step5: fine tune
    print("Fine tuning---------------")
    model1.layers[2].trainable = True

    # It's important to recompile your model after you make any changes
    optimizer = tf.keras.optimizers.Adam(learning_rate_fine_tuning)
    loss = tf.keras.losses.CategoricalCrossentropy(name='categorical_crossentropy')#from_logits=False,label_smoothing=0.0,axis=-1,
    metrics = []
    metrics.append(
        tf.metrics.CategoricalAccuracy(name='categorical_accuracy', dtype=None))
    model1.compile(optimizer=optimizer, loss= loss, metrics=metrics)

    history = model1.fit(train, validation_data=val, epochs=6, class_weight=my_weight2 ,callbacks=[ early_stopping])
    balanced_acc, McNemar=get_balanced_accuracy(model1, McNemar, True, drop_out_rate,learning_rate_transfer_learning, learning_rate_fine_tuning )
    balanced_accuracies_fine_tuning.append( balanced_acc )
    print("----------------------------------------")
    del(model1)


learning_rate_transfer_learning:  0.001 learning_rate_fine_tuning:  1e-05 drop_out_rate: 0.3


Some layers from the model checkpoint at xlnet-base-cased were not used when initializing TFXLNetModel: ['lm_loss']
- This IS expected if you are initializing TFXLNetModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFXLNetModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFXLNetModel were initialized from the model checkpoint at xlnet-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFXLNetModel for predictions without further training.


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 768)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 768)]        0           []                               
                                                                                                  
 tfxl_net_model (TFXLNetModel)  TFXLNetModelOutput(  116718336   ['input_ids[0][0]',              
                                last_hidden_state=(               'attention_mask[0][0]']         
                                None, 768, 768),                                                  
                                 mems=((768, None,                                            

2437/2437 [==============================] - 1726s 702ms/step - loss: 2.0378 - categorical_accuracy: 0.2391 - val_loss: 3.1614 - val_categorical_accuracy: 0.1577
Epoch 2/6
2437/2437 [==============================] - 1709s 701ms/step - loss: 1.9646 - categorical_accuracy: 0.2573 - val_loss: 16.1496 - val_categorical_accuracy: 0.0517
Epoch 3/6
2437/2437 [==============================] - 1710s 701ms/step - loss: 1.9279 - categorical_accuracy: 0.2680 - val_loss: 11.6222 - val_categorical_accuracy: 0.0701
Epoch 4/6
2437/2437 [==============================] - 1710s 702ms/step - loss: 1.9098 - categorical_accuracy: 0.2625 - val_loss: 16.3768 - val_categorical_accuracy: 0.0517


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classificat

...

Some layers from the model checkpoint at xlnet-base-cased were not used when initializing TFXLNetModel: ['lm_loss']
- This IS expected if you are initializing TFXLNetModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFXLNetModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFXLNetModel were initialized from the model checkpoint at xlnet-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFXLNetModel for predictions without further training.


Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 768)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 768)]        0           []                               
                                                                                                  
 tfxl_net_model_3 (TFXLNetModel  TFXLNetModelOutput(  116718336  ['input_ids[0][0]',              
 )                              last_hidden_state=(               'attention_mask[0][0]']         
                                None, 768, 768),                                                  
                                 mems=((768, None,                                          

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classificat

2437/2437 [==============================] - 1738s 707ms/step - loss: 2.1887 - categorical_accuracy: 0.1947 - val_loss: 2.4011 - val_categorical_accuracy: 0.1614
Epoch 2/6
2437/2437 [==============================] - 1720s 706ms/step - loss: 2.2143 - categorical_accuracy: 0.1808 - val_loss: 2.3591 - val_categorical_accuracy: 0.1485
Epoch 3/6
2437/2437 [==============================] - 1721s 706ms/step - loss: 2.1973 - categorical_accuracy: 0.1867 - val_loss: 2.4682 - val_categorical_accuracy: 0.1448
Epoch 4/6
2437/2437 [==============================] - 1722s 707ms/step - loss: 2.3465 - categorical_accuracy: 0.1444 - val_loss: 3.4109 - val_categorical_accuracy: 0.0895
              precision    recall  f1-score   support

         0.0       0.07      0.82      0.14       190
         1.0       0.00      0.00      0.00       173
         2.0       0.00      0.00      0.00       135
         3.0       1.00      0.00      0.01       216
         4.0       0.37      0.02      0.04      10

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classificat

In [ ]:
from transformers import TFXLNetModel


#drop_out_rates = [0.1, 0.2, 0.3,0.4, 0.5]
balanced_accuracies_transfer_learning = []
balanced_accuracies_fine_tuning = []

drop_out_rate = 0.4
learning_rate_transfer_learnings = [1e-3, 1e-4]
learning_rate_fine_tunings = [1e-5, 1e-6]

for learning_rate_transfer_learning in learning_rate_transfer_learnings:
  for learning_rate_fine_tuning in learning_rate_fine_tunings:
    print("learning_rate_transfer_learning: ",learning_rate_transfer_learning, "learning_rate_fine_tuning: "
    , learning_rate_fine_tuning, "drop_out_rate:", drop_out_rate)
    #step1
    #bert_base = TFAutoModel.from_pretrained('bert-base-cased')
    xlnet_base = TFXLNetModel.from_pretrained('xlnet-base-cased')
    input_ids = tf.keras.layers.Input(shape=(SEQ_LEN2,), name= 'input_ids', dtype='int32')
    mask = tf.keras.layers.Input(shape=(SEQ_LEN2,), name='attention_mask')

    embeddings = xlnet_base(input_ids, attention_mask= mask)[0]
    X = tf.keras.layers.GlobalMaxPool1D()(embeddings)
    X = tf.keras.layers.BatchNormalization()(X)
    X = tf.keras.layers.Dense(128, activation = 'relu')(X)
    X = tf.keras.layers.BatchNormalization()(X)#added
    X = tf.keras.layers.Dropout(drop_out_rate)(X)
    X = tf.keras.layers.Dense(32, activation = 'relu')(X)
    X = tf.keras.layers.BatchNormalization()(X)#added
    X = tf.keras.layers.Dropout(drop_out_rate)(X)
    y= tf.keras.layers.Dense(10, activation = 'softmax' , name= 'outputs')(X)

    model1 = tf.keras.Model(inputs=[input_ids, mask], outputs=y)
    model1.layers[2].trainable = False
    print(model1.summary())

    #step2
    optimizer = tf.keras.optimizers.Adam(learning_rate_transfer_learning)
    loss = tf.keras.losses.CategoricalCrossentropy(name='categorical_crossentropy')#from_logits=False,label_smoothing=0.0,axis=-1,
    metrics = []
    metrics.append(
        tf.metrics.CategoricalAccuracy(name='categorical_accuracy', dtype=None))
    model1.compile(optimizer=optimizer, loss= loss, metrics=metrics)
    #model2.summary() #Check trainable params increased.

    #step3: transfer learning
    checkpoint_filepath = DIR + 'checkpoints'
    model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_filepath, save_weights_only=True
                                                  , monitor='val_categorical_accuracy', save_best_only=False)#, mode='max'
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_categorical_accuracy', patience=3)
    history = model1.fit(train, validation_data=val, epochs=6, class_weight=my_weight2 ,callbacks=[ early_stopping, model_checkpoint_callback])

    #step4: predict
    balanced_acc, McNemar=get_balanced_accuracy(model1, McNemar, False, drop_out_rate,learning_rate_transfer_learning, learning_rate_fine_tuning )
    balanced_accuracies_transfer_learning.append( balanced_acc )

    #step5: fine tune
    print("Fine tuning---------------")
    model1.layers[2].trainable = True

    # It's important to recompile your model after you make any changes
    optimizer = tf.keras.optimizers.Adam(learning_rate_fine_tuning)
    loss = tf.keras.losses.CategoricalCrossentropy(name='categorical_crossentropy')#from_logits=False,label_smoothing=0.0,axis=-1,
    metrics = []
    metrics.append(
        tf.metrics.CategoricalAccuracy(name='categorical_accuracy', dtype=None))
    model1.compile(optimizer=optimizer, loss= loss, metrics=metrics)

    history = model1.fit(train, validation_data=val, epochs=6, class_weight=my_weight2 ,callbacks=[ early_stopping])
    balanced_acc, McNemar=get_balanced_accuracy(model1, McNemar, True, drop_out_rate,learning_rate_transfer_learning, learning_rate_fine_tuning )
    balanced_accuracies_fine_tuning.append( balanced_acc )
    print("----------------------------------------")
    del(model1)


learning_rate_transfer_learning:  0.001 learning_rate_fine_tuning:  1e-05 drop_out_rate: 0.4


Some layers from the model checkpoint at xlnet-base-cased were not used when initializing TFXLNetModel: ['lm_loss']
- This IS expected if you are initializing TFXLNetModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFXLNetModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFXLNetModel were initialized from the model checkpoint at xlnet-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFXLNetModel for predictions without further training.


Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 768)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 768)]        0           []                               
                                                                                                  
 tfxl_net_model_4 (TFXLNetModel  TFXLNetModelOutput(  116718336  ['input_ids[0][0]',              
 )                              last_hidden_state=(               'attention_mask[0][0]']         
                                None, 768, 768),                                                  
                                 mems=((768, None,                                          

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classificat

2437/2437 [==============================] - 1736s 706ms/step - loss: 2.0440 - categorical_accuracy: 0.2287 - val_loss: 3.5079 - val_categorical_accuracy: 0.1458
Epoch 2/6
2437/2437 [==============================] - 1720s 706ms/step - loss: 2.0329 - categorical_accuracy: 0.2337 - val_loss: 14.9063 - val_categorical_accuracy: 0.1541
Epoch 3/6
2437/2437 [==============================] - 1721s 706ms/step - loss: 1.9825 - categorical_accuracy: 0.2504 - val_loss: 8.6482 - val_categorical_accuracy: 0.1522
Epoch 4/6
2437/2437 [==============================] - 1721s 706ms/step - loss: 1.9711 - categorical_accuracy: 0.2612 - val_loss: 13.4391 - val_categorical_accuracy: 0.0701
Epoch 5/6
2437/2437 [==============================] - 1721s 706ms/step - loss: 1.9536 - categorical_accuracy: 0.2442 - val_loss: 7.5646 - val_categorical_accuracy: 0.1494


...

              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00       190
         1.0       0.00      0.00      0.00       173
         2.0       0.00      0.00      0.00       135
         3.0       0.00      0.00      0.00       216
         4.0       0.00      0.00      0.00      1036
         5.0       0.43      0.04      0.07        76
         6.0       0.85      0.70      0.77       195
         7.0       0.00      0.00      0.00       290
         8.0       0.00      0.00      0.00       139
         9.0       0.10      0.98      0.18       258

    accuracy                           0.15      2708
   macro avg       0.14      0.17      0.10      2708
weighted avg       0.08      0.15      0.07      2708

0.17226579418133886
----------------------------------------
learning_rate_transfer_learning:  0.001 learning_rate_fine_tuning:  1e-06 drop_out_rate: 0.4


Some layers from the model checkpoint at xlnet-base-cased were not used when initializing TFXLNetModel: ['lm_loss']
- This IS expected if you are initializing TFXLNetModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFXLNetModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFXLNetModel were initialized from the model checkpoint at xlnet-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFXLNetModel for predictions without further training.


Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 768)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 768)]        0           []                               
                                                                                                  
 tfxl_net_model_5 (TFXLNetModel  TFXLNetModelOutput(  116718336  ['input_ids[0][0]',              
 )                              last_hidden_state=(               'attention_mask[0][0]']         
                                None, 768, 768),                                                  
                                 mems=((768, None,                                          

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classificat

2332/2437 [===========================>..] - ETA: 1:12 - loss: 2.0250 - categorical_accuracy: 0.2639

In [ ]:
from transformers import TFXLNetModel


#drop_out_rates = [0.1, 0.2, 0.3,0.4, 0.5]
balanced_accuracies_transfer_learning = []
balanced_accuracies_fine_tuning = []

drop_out_rate = 0.5
learning_rate_transfer_learnings = [1e-3, 1e-4]
learning_rate_fine_tunings = [1e-5, 1e-6]

for learning_rate_transfer_learning in learning_rate_transfer_learnings:
  for learning_rate_fine_tuning in learning_rate_fine_tunings:
    print("learning_rate_transfer_learning: ",learning_rate_transfer_learning, "learning_rate_fine_tuning: "
    , learning_rate_fine_tuning, "drop_out_rate:", drop_out_rate)
    #step1
    #bert_base = TFAutoModel.from_pretrained('bert-base-cased')
    xlnet_base = TFXLNetModel.from_pretrained('xlnet-base-cased')
    input_ids = tf.keras.layers.Input(shape=(SEQ_LEN2,), name= 'input_ids', dtype='int32')
    mask = tf.keras.layers.Input(shape=(SEQ_LEN2,), name='attention_mask')

    embeddings = xlnet_base(input_ids, attention_mask= mask)[0]
    X = tf.keras.layers.GlobalMaxPool1D()(embeddings)
    X = tf.keras.layers.BatchNormalization()(X)
    X = tf.keras.layers.Dense(128, activation = 'relu')(X)
    X = tf.keras.layers.BatchNormalization()(X)#added
    X = tf.keras.layers.Dropout(drop_out_rate)(X)
    X = tf.keras.layers.Dense(32, activation = 'relu')(X)
    X = tf.keras.layers.BatchNormalization()(X)#added
    X = tf.keras.layers.Dropout(drop_out_rate)(X)
    y= tf.keras.layers.Dense(10, activation = 'softmax' , name= 'outputs')(X)

    model1 = tf.keras.Model(inputs=[input_ids, mask], outputs=y)
    model1.layers[2].trainable = False
    print(model1.summary())

    #step2
    optimizer = tf.keras.optimizers.Adam(learning_rate_transfer_learning)
    loss = tf.keras.losses.CategoricalCrossentropy(name='categorical_crossentropy')#from_logits=False,label_smoothing=0.0,axis=-1,
    metrics = []
    metrics.append(
        tf.metrics.CategoricalAccuracy(name='categorical_accuracy', dtype=None))
    model1.compile(optimizer=optimizer, loss= loss, metrics=metrics)
    #model2.summary() #Check trainable params increased.

    #step3: transfer learning
    checkpoint_filepath = DIR + 'checkpoints'
    model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_filepath, save_weights_only=True
                                                  , monitor='val_categorical_accuracy', save_best_only=False)#, mode='max'
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_categorical_accuracy', patience=3)
    history = model1.fit(train, validation_data=val, epochs=6, class_weight=my_weight2 ,callbacks=[ early_stopping, model_checkpoint_callback])

    #step4: predict
    balanced_acc, McNemar=get_balanced_accuracy(model1, McNemar, False, drop_out_rate,learning_rate_transfer_learning, learning_rate_fine_tuning )
    balanced_accuracies_transfer_learning.append( balanced_acc )

    #step5: fine tune
    print("Fine tuning---------------")
    model1.layers[2].trainable = True

    # It's important to recompile your model after you make any changes
    optimizer = tf.keras.optimizers.Adam(learning_rate_fine_tuning)
    loss = tf.keras.losses.CategoricalCrossentropy(name='categorical_crossentropy')#from_logits=False,label_smoothing=0.0,axis=-1,
    metrics = []
    metrics.append(
        tf.metrics.CategoricalAccuracy(name='categorical_accuracy', dtype=None))
    model1.compile(optimizer=optimizer, loss= loss, metrics=metrics)

    history = model1.fit(train, validation_data=val, epochs=6, class_weight=my_weight2 ,callbacks=[ early_stopping])
    balanced_acc, McNemar=get_balanced_accuracy(model1, McNemar, True, drop_out_rate,learning_rate_transfer_learning, learning_rate_fine_tuning )
    balanced_accuracies_fine_tuning.append( balanced_acc )
    print("----------------------------------------")
    del(model1)


In [ ]:
#how to use checkpoint model
#model.load_weights(checkpoint_filepath)

### Try smaller learning rate only transfer learning

In [ ]:
McNemar={}

In [ ]:
from transformers import TFXLNetModel


drop_out_rates = [0.2, 0.3,0.4, 0.5]
balanced_accuracies_transfer_learning = []
balanced_accuracies_fine_tuning = []

#drop_out_rate = 0.2
learning_rate_transfer_learnings = [1e-6]
#learning_rate_fine_tunings = [1e-5, 1e-6]
learning_rate_fine_tuning=1e-5

for drop_out_rate in drop_out_rates:
  for learning_rate_transfer_learning in learning_rate_transfer_learnings:
    print("learning_rate_transfer_learning: ",learning_rate_transfer_learning, "drop_out_rate:", drop_out_rate)
    #step1
    #bert_base = TFAutoModel.from_pretrained('bert-base-cased')
    xlnet_base = TFXLNetModel.from_pretrained('xlnet-base-cased')
    input_ids = tf.keras.layers.Input(shape=(SEQ_LEN2,), name= 'input_ids', dtype='int32')
    mask = tf.keras.layers.Input(shape=(SEQ_LEN2,), name='attention_mask')

    embeddings = xlnet_base(input_ids, attention_mask= mask)[0]
    X = tf.keras.layers.GlobalMaxPool1D()(embeddings)
    X = tf.keras.layers.BatchNormalization()(X)
    X = tf.keras.layers.Dense(128, activation = 'relu')(X)
    X = tf.keras.layers.BatchNormalization()(X)#added
    X = tf.keras.layers.Dropout(drop_out_rate)(X)
    X = tf.keras.layers.Dense(32, activation = 'relu')(X)
    X = tf.keras.layers.BatchNormalization()(X)#added
    X = tf.keras.layers.Dropout(drop_out_rate)(X)
    y= tf.keras.layers.Dense(10, activation = 'softmax' , name= 'outputs')(X)

    model1 = tf.keras.Model(inputs=[input_ids, mask], outputs=y)
    model1.layers[2].trainable = False
    print(model1.summary())

    #step2
    optimizer = tf.keras.optimizers.Adam(learning_rate_transfer_learning)
    loss = tf.keras.losses.CategoricalCrossentropy(name='categorical_crossentropy')#from_logits=False,label_smoothing=0.0,axis=-1,
    metrics = []
    metrics.append(
        tf.metrics.CategoricalAccuracy(name='categorical_accuracy', dtype=None))
    model1.compile(optimizer=optimizer, loss= loss, metrics=metrics)
    #model2.summary() #Check trainable params increased.

    #step3: transfer learning
    checkpoint_filepath = DIR + 'checkpoints'
    model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_filepath, save_weights_only=True
                                                  , monitor='val_categorical_accuracy', save_best_only=False)#, mode='max'
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_categorical_accuracy', patience=3)
    history = model1.fit(train, validation_data=val, epochs=6, class_weight=my_weight2 ,callbacks=[ early_stopping, model_checkpoint_callback])

    #step4: predict
    balanced_acc, McNemar=get_balanced_accuracy(model1, McNemar, False, drop_out_rate,learning_rate_transfer_learning, learning_rate_fine_tuning )
    balanced_accuracies_transfer_learning.append( balanced_acc )

    del(model1)


learning_rate_transfer_learning:  1e-06 drop_out_rate: 0.2


Some layers from the model checkpoint at xlnet-base-cased were not used when initializing TFXLNetModel: ['lm_loss']
- This IS expected if you are initializing TFXLNetModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFXLNetModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFXLNetModel were initialized from the model checkpoint at xlnet-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFXLNetModel for predictions without further training.


Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 768)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 768)]        0           []                               
                                                                                                  
 tfxl_net_model_1 (TFXLNetModel  TFXLNetModelOutput(  116718336  ['input_ids[0][0]',              
 )                              last_hidden_state=(               'attention_mask[0][0]']         
                                None, 768, 768),                                                  
                                 mems=((768, None,                                          

Some layers from the model checkpoint at xlnet-base-cased were not used when initializing TFXLNetModel: ['lm_loss']
- This IS expected if you are initializing TFXLNetModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFXLNetModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFXLNetModel were initialized from the model checkpoint at xlnet-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFXLNetModel for predictions without further training.


Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 768)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 768)]        0           []                               
                                                                                                  
 tfxl_net_model_2 (TFXLNetModel  TFXLNetModelOutput(  116718336  ['input_ids[0][0]',              
 )                              last_hidden_state=(               'attention_mask[0][0]']         
                                None, 768, 768),                                                  
                                 mems=((768, None,                                          

Some layers from the model checkpoint at xlnet-base-cased were not used when initializing TFXLNetModel: ['lm_loss']
- This IS expected if you are initializing TFXLNetModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFXLNetModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFXLNetModel were initialized from the model checkpoint at xlnet-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFXLNetModel for predictions without further training.


Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 768)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 768)]        0           []                               
                                                                                                  
 tfxl_net_model_3 (TFXLNetModel  TFXLNetModelOutput(  116718336  ['input_ids[0][0]',              
 )                              last_hidden_state=(               'attention_mask[0][0]']         
                                None, 768, 768),                                                  
                                 mems=((768, None,                                          

Some layers from the model checkpoint at xlnet-base-cased were not used when initializing TFXLNetModel: ['lm_loss']
- This IS expected if you are initializing TFXLNetModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFXLNetModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFXLNetModel were initialized from the model checkpoint at xlnet-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFXLNetModel for predictions without further training.


Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 768)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 768)]        0           []                               
                                                                                                  
 tfxl_net_model_4 (TFXLNetModel  TFXLNetModelOutput(  116718336  ['input_ids[0][0]',              
 )                              last_hidden_state=(               'attention_mask[0][0]']         
                                None, 768, 768),                                                  
                                 mems=((768, None,                                          

In [ ]:
from transformers import TFXLNetModel


drop_out_rates = [0.2, 0.3,0.4, 0.5]
balanced_accuracies_transfer_learning = []
balanced_accuracies_fine_tuning = []

#drop_out_rate = 0.2
learning_rate_transfer_learnings = [1e-5, 1e-6]
#learning_rate_fine_tunings = [1e-5, 1e-6]
learning_rate_fine_tuning=1e-5

for drop_out_rate in drop_out_rates:
  for learning_rate_transfer_learning in learning_rate_transfer_learnings:
    print("learning_rate_transfer_learning: ",learning_rate_transfer_learning, "drop_out_rate:", drop_out_rate)
    #step1
    #bert_base = TFAutoModel.from_pretrained('bert-base-cased')
    xlnet_base = TFXLNetModel.from_pretrained('xlnet-base-cased')
    input_ids = tf.keras.layers.Input(shape=(SEQ_LEN2,), name= 'input_ids', dtype='int32')
    mask = tf.keras.layers.Input(shape=(SEQ_LEN2,), name='attention_mask')

    embeddings = xlnet_base(input_ids, attention_mask= mask)[0]
    X = tf.keras.layers.GlobalMaxPool1D()(embeddings)
    X = tf.keras.layers.BatchNormalization()(X)
    X = tf.keras.layers.Dense(512, activation = 'relu')(X)
    X = tf.keras.layers.BatchNormalization()(X)#added
    X = tf.keras.layers.Dropout(drop_out_rate)(X)
    X = tf.keras.layers.Dense(512, activation = 'relu')(X)
    X = tf.keras.layers.BatchNormalization()(X)#added
    X = tf.keras.layers.Dropout(drop_out_rate)(X)
    y= tf.keras.layers.Dense(10, activation = 'softmax' , name= 'outputs')(X)

    model1 = tf.keras.Model(inputs=[input_ids, mask], outputs=y)
    model1.layers[2].trainable = False
    print(model1.summary())

    #step2
    optimizer = tf.keras.optimizers.Adam(learning_rate_transfer_learning)
    loss = tf.keras.losses.CategoricalCrossentropy(name='categorical_crossentropy')#from_logits=False,label_smoothing=0.0,axis=-1,
    metrics = []
    metrics.append(
        tf.metrics.CategoricalAccuracy(name='categorical_accuracy', dtype=None))
    model1.compile(optimizer=optimizer, loss= loss, metrics=metrics)
    #model2.summary() #Check trainable params increased.

    #step3: transfer learning
    checkpoint_filepath = DIR + 'checkpoints'
    model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_filepath, save_weights_only=True
                                                  , monitor='val_categorical_accuracy', save_best_only=False)#, mode='max'
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_categorical_accuracy', patience=3)
    history = model1.fit(train, validation_data=val, epochs=6, class_weight=my_weight2 ,callbacks=[ early_stopping, model_checkpoint_callback])

    #step4: predict
    balanced_acc, McNemar=get_balanced_accuracy(model1, McNemar, False, drop_out_rate,learning_rate_transfer_learning, learning_rate_fine_tuning )
    balanced_accuracies_transfer_learning.append( balanced_acc )

    del(model1)


learning_rate_transfer_learning:  1e-05 drop_out_rate: 0.2


Some layers from the model checkpoint at xlnet-base-cased were not used when initializing TFXLNetModel: ['lm_loss']
- This IS expected if you are initializing TFXLNetModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFXLNetModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFXLNetModel were initialized from the model checkpoint at xlnet-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFXLNetModel for predictions without further training.


Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 768)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 768)]        0           []                               
                                                                                                  
 tfxl_net_model_5 (TFXLNetModel  TFXLNetModelOutput(  116718336  ['input_ids[0][0]',              
 )                              last_hidden_state=(               'attention_mask[0][0]']         
                                None, 768, 768),                                                  
                                 mems=((768, None,                                          

Some layers from the model checkpoint at xlnet-base-cased were not used when initializing TFXLNetModel: ['lm_loss']
- This IS expected if you are initializing TFXLNetModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFXLNetModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFXLNetModel were initialized from the model checkpoint at xlnet-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFXLNetModel for predictions without further training.


Model: "model_6"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 768)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 768)]        0           []                               
                                                                                                  
 tfxl_net_model_6 (TFXLNetModel  TFXLNetModelOutput(  116718336  ['input_ids[0][0]',              
 )                              last_hidden_state=(               'attention_mask[0][0]']         
                                None, 768, 768),                                                  
                                 mems=((768, None,                                          

...

Model: "model_10"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 768)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 768)]        0           []                               
                                                                                                  
 tfxl_net_model_10 (TFXLNetMode  TFXLNetModelOutput(  116718336  ['input_ids[0][0]',              
 l)                             last_hidden_state=(               'attention_mask[0][0]']         
                                None, 768, 768),                                                  
                                 mems=((768, None,                                         

Some layers from the model checkpoint at xlnet-base-cased were not used when initializing TFXLNetModel: ['lm_loss']
- This IS expected if you are initializing TFXLNetModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFXLNetModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFXLNetModel were initialized from the model checkpoint at xlnet-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFXLNetModel for predictions without further training.


Model: "model_11"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 768)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 768)]        0           []                               
                                                                                                  
 tfxl_net_model_11 (TFXLNetMode  TFXLNetModelOutput(  116718336  ['input_ids[0][0]',              
 l)                             last_hidden_state=(               'attention_mask[0][0]']         
                                None, 768, 768),                                                  
                                 mems=((768, None,                                         

Some layers from the model checkpoint at xlnet-base-cased were not used when initializing TFXLNetModel: ['lm_loss']
- This IS expected if you are initializing TFXLNetModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFXLNetModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFXLNetModel were initialized from the model checkpoint at xlnet-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFXLNetModel for predictions without further training.


Model: "model_12"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 768)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 768)]        0           []                               
                                                                                                  
 tfxl_net_model_12 (TFXLNetMode  TFXLNetModelOutput(  116718336  ['input_ids[0][0]',              
 l)                             last_hidden_state=(               'attention_mask[0][0]']         
                                None, 768, 768),                                                  
                                 mems=((768, None,                                         